In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = de5d43e
molecule = {'h2o': 'atmpro', 'co2': 0.0004}
band = [11]
atmpro = saw
tsfc = 257


...

Best-fit values for each (gas, band)
------------------------------------
o h2o band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(500, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66)]
  wgt = [(0.4, 0.4, 0.5, 0.55, 0.85)]
o co2 band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [3, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.75), (1.75, 1.6, 1.85)]
  wgt = [(0.7, 0.8, 0.7), (0.8, 0.7, 0.8)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.001222                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.079071
0.000750      2.0 -0.156257                  2.0 -0.141173
0.001052      3.0 -0.113086                  3.0 -0.100443
0.001476      4.0 -0.091351                  4.0 -0.080486
0.002070      5.0 -0.074332                  5.0 -0.078248
0.002904      6.0 -0.057947                  6.0 -0.081449
0.004074      7.0 -0.041303                  7.0 -0.081252
0.005714      8.0 -0.024418                  8.0 -0.071980
0.008015      9.0 -0.007762                  9.0 -0.053405
0.011243     10.0  0.008256                 10.0 -0.026741
0.015771     11.0  0.023563                 11.0  0.005440
0.022122     12.0  0.039150                 12.0  0.042242
0.031031     13.0  0.060655                 13.0  0.086632
0.043528     14.0  0.076175                 14.0  0.122480
0.061057     15.0  0.073413                 15.0  0.133050
0.085645     16.0  0.068762                 16.0  0.128203
0.120136     17.0  0.070392                 17.0  0.116408
0.168516     18.0  0.075295                 18.0  0.098641
0.236378     19.0  0.083674                 19.0  0.081628
0.331549     20.0  0.098614                 20.0  0.076586
0.465100     21.0  0.134445                 21.0  0.107313
0.652400     22.0  0.121463                 22.0  0.097295
0.915100     23.0  0.063170                 23.0  0.051727
1.283650     24.0  0.030823                 24.0  0.035179
1.800600     25.0  0.016823                 25.0  0.027765
2.525700     26.0  0.009029                 26.0  0.019515
3.542800     27.0  0.003926                 27.0  0.010585
4.969550     28.0  0.001656                 28.0  0.003368
6.970850     29.0  0.001418                 29.0  0.000430
9.778100     30.0  0.001198                 30.0  0.000724
13.715850    31.0  0.000618                 31.0  0.001405
19.239350    32.0 -0.000109                 32.0  0.000903
26.987250    33.0 -0.000258                 33.0  0.000429
37.855300    34.0  0.000047                 34.0  0.000317
53.100050    35.0  0.000133                 35.0  0.000019
73.887500    36.0  0.000160                 36.0 -0.000086
97.662500    37.0  0.000183                 37.0 -0.000044
121.437500   38.0  0.000042                 38.0 -0.000181
145.212500   39.0 -0.000144                 39.0 -0.000364
168.987500   40.0 -0.000257                 40.0 -0.000461
192.762500   41.0 -0.000365                 41.0 -0.000571
216.537500   42.0 -0.000483                 42.0 -0.000702
240.312500   43.0 -0.000623                 43.0 -0.000854
264.087500   44.0 -0.000840                 44.0 -0.001078
287.862500   45.0 -0.001303                 45.0 -0.001583
311.637500   46.0 -0.001621                 46.0 -0.001935
335.412500   47.0 -0.001419                 47.0 -0.001733
359.187500   48.0 -0.001116                 48.0 -0.001465
382.962500   49.0 -0.000943                 49.0 -0.001341
406.737500   50.0 -0.000749                 50.0 -0.001191
430.512500   51.0 -0.000507                 51.0 -0.000932
454.287500   52.0 -0.000205                 52.0 -0.000588
478.062500   53.0  0.000169                 53.0 -0.000166
501.837500   54.0  0.000622                 54.0  0.000256
525.612500   55.0  0.001159                 55.0  0.000683
549.387500   56.0  0.001774                 56.0  0.001163
573.162500   57.0  0.002456                 57.0  0.001762
596.937500   58.0  0.003183                 58.0  0.002511
620.712500   59.0  0.003924                 59.0  0.003350
644.487500   60.0  0.004649                 60.0  0.004288
668.262500   61.0  0.005332                 61.0  0.005168
692.037500   62.0  0.005958                 62.0  0.006054
715.812500   63.0  0.006531                 63.0  0.006830
739.587500   64.0  0.007074                 64.0  0.007535
76

# Fluxes by Layer

CRD                        CLIRAD                    
                        flug      fldg     fnetg      flug      fldg     fnetg
pressure     level                                                            
0.000000e+00 1     -1.164942  0.000000 -1.164942       NaN       NaN       NaN
1.000000e-08 1           NaN       NaN       NaN -1.146152  0.000002 -1.146150
6.244000e-04 2     -1.164957  0.000016 -1.164942 -1.146175  0.000031 -1.146144
8.759000e-04 3     -1.164966  0.000019 -1.164946 -1.146187  0.000038 -1.146149
1.228600e-03 4     -1.164976  0.000025 -1.164951 -1.146202  0.000049 -1.146153
1.723400e-03 5     -1.164991  0.000034 -1.164956 -1.146222  0.000064 -1.146158
2.417400e-03 6     -1.165010  0.000048 -1.164963 -1.146247  0.000083 -1.146164
3.390900e-03 7     -1.165036  0.000067 -1.164969 -1.146280  0.000107 -1.146173
4.756500e-03 8     -1.165069  0.000093 -1.164976 -1.146323  0.000137 -1.146186
6.672000e-03 9     -1.165113  0.000131 -1.164982 -1.146379  0.000176 -1.146203
9.358900e-03 10    -1.165169  0.000185 -1.164984 -1.146449  0.000229 -1.146220
1.312780e-02 11    -1.165241  0.000261 -1.164980 -1.146538  0.000306 -1.146232
1.841450e-02 12    -1.165331  0.000366 -1.164966 -1.146648  0.000420 -1.146228
2.583020e-02 13    -1.165441  0.000510 -1.164931 -1.146780  0.000589 -1.146191
3.623230e-02 14    -1.165564  0.000708 -1.164856 -1.146928  0.000844 -1.146084
5.082340e-02 15    -1.165688  0.000963 -1.164725 -1.147082  0.001210 -1.145873
7.129060e-02 16    -1.165818  0.001271 -1.164547 -1.147250  0.001699 -1.145550
1.000000e-01 17    -1.165961  0.001649 -1.164313 -1.147439  0.002325 -1.145114
1.402710e-01 18    -1.166111  0.002134 -1.163977 -1.147644  0.003085 -1.144559
1.967600e-01 19    -1.166246  0.002773 -1.163473 -1.147850  0.003951 -1.143899
2.759970e-01 20    -1.166317  0.003629 -1.162688 -1.148020  0.004888 -1.143132
3.871000e-01 21    -1.166185  0.004795 -1.161390 -1.148029  0.005904 -1.142124
5.431000e-01 22    -1.165267  0.006361 -1.158906 -1.147211  0.007070 -1.140141
7.617000e-01 23    -1.163442  0.007682 -1.155760 -1.145385  0.007764 -1.137621
1.068500e+00 24    -1.161537  0.008072 -1.153465 -1.143462  0.007721 -1.135741
1.498800e+00 25    -1.159850  0.007956 -1.151893 -1.141744  0.007796 -1.133948
2.102400e+00 26    -1.158361  0.007670 -1.150691 -1.140178  0.008216 -1.131962
2.949000e+00 27    -1.157071  0.007286 -1.149785 -1.138758  0.008752 -1.130005
4.136600e+00 28    -1.156061  0.006828 -1.149233 -1.137560  0.009045 -1.128516
5.802500e+00 29    -1.155338  0.006432 -1.148906 -1.136632  0.008780 -1.127851
8.139200e+00 30    -1.154754  0.006240 -1.148514 -1.135828  0.008095 -1.127732
1.141700e+01 31    -1.154242  0.006194 -1.148048 -1.135074  0.007623 -1.127451
1.601470e+01 32    -1.153887  0.006175 -1.147712 -1.134505  0.007819 -1.126686
2.246400e+01 33    -1.153979  0.006184 -1.147795 -1.134523  0.008527 -1.125996
3.151050e+01 34    -1.154657  0.006586 -1.148071 -1.135280  0.009744 -1.125536
4.420010e+01 35    -1.155709  0.007708 -1.148001 -1.136487  0.011426 -1.125061
6.200000e+01 36    -1.157106  0.009387 -1.147719 -1.138124  0.013104 -1.125020
8.577500e+01 37    -1.158742  0.011473 -1.147269 -1.140030  0.014769 -1.125262
1.095500e+02 38    -1.160111  0.013359 -1.146752 -1.141660  0.016274 -1.125386
1.333250e+02 39    -1.161391  0.014758 -1.146634 -1.143244  0.017348 -1.125896
1.571000e+02 40    -1.162856  0.015815 -1.147040 -1.145124  0.018204 -1.126920
1.808750e+02 41    -1.164537  0.016774 -1.147763 -1.147337  0.019120 -1.128217
2.046500e+02 42    -1.166472  0.017680 -1.148793 -1.149917  0.020092 -1.129824
2.284250e+02 43    -1.168695  0.018541 -1.150154 -1.152903  0.021101 -1.131802
2.522000e+02 44    -1.171278  0.019370 -1.151908 -1.156342  0.022133 -1.134208
2.759750e+02 45    -1.174458  0.020186 -1.154273 -1.160435  0.023190 -1.137245
2.997500e+02 46    -1.179076  0.021133 -1.157943 -1.166126  0.024423 -1.141704
3.235250e+02 47    -1.185636  0.023129 -1.162507 -1.173995  0.026840 -1.147

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')